In [2]:
import numpy as np
import pandas as pd 

In [11]:
DATA_IN_PATH = './data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA, header=0, delimiter=',',quoting=3) # quoting NONE


reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

## TF-IDF 벡터화

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

"""
min_df : 벡터화 과정 포함될 최소 df값
analyzer: char(문자), word(단어) 중 선택,단위 분석 옵션
sublinear_tf : 문서의 단어 빈도 수에 대한 스무딩(smoothing) 여부 설정
ngram_range: 빈도 기본 단위 설정
max_features: 각 벡터의 최대 길이 즉, 특징의 길이를 설정
"""

vectorizer =TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=5000)


X = vectorizer.fit_transform(reviews)

### 검증 데이터셋 분리

In [17]:
from sklearn.model_selection import train_test_split
import numpy as np

RANDOM_SEED = 42
TEST_SPLIT = 0.2

y = np.array(sentiments) #정답 라벨의 넘파이 배열
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)



In [19]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight = 'balanced') #default reg = l2
lgs.fit(X_train,y_train)

LogisticRegression(class_weight='balanced')

In [24]:
print("Accuracy: {:0.5f}".format(lgs.score(X_eval, y_eval)))

Accuracy: 0.85980


### 데이터 캐글에 제출하기

In [35]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA, header=0, delimiter=',', quoting=3)

testDataVecs = vectorizer.transform(test_data['review'])
test_predicted = lgs.predict(testDataVecs)

print(test_predicted)

[1 0 1 ... 0 1 0]


In [43]:
test_data = test_data.apply(lambda x : x.str.strip("\"")) # id에 quote를 제거함

print(test_data['id'])

0        12311_10
1          8348_2
2          5828_4
3          7186_2
4         12128_7
           ...   
24995     2155_10
24996       59_10
24997      2531_1
24998      7772_8
24999    11465_10
Name: id, Length: 25000, dtype: object


In [49]:
## 캐글 제출시 CSV파일로 저장하여 제출해야함
## 형식 : 각 데이터 id와 결괏값
import os

DATA_OUT_PATH = './data_out/'

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)


ids = list(test_data['id'])
 ## 답과 id를 통해 데이터 프레임 생성
answer_dataset = pd.DataFrame({'id':ids, 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv',index=False) ##인덱스 제거해줘야함!
